In [1]:
from IPython.core.display import HTML
HTML('''
<script>
var logoParent = document.getElementById("kernel_logo_widget")
var logo = document.getElementById("kernel_logo_widget").getElementsByClassName("current_kernel_logo")[0];
logo.src = "https://i.ibb.co/mD4jTGQ/itclogo.jpg";
logo.style = "display: inline; width:138px; height:40px";
logoParent.innerHTML = '<a href="https://i.ibb.co/mD4jTGQ/itclogo.jpg">' + logoParent.innerHTML + '</a>';
</script>
''')

<font size=10><b>Feature Selection - Exercise</b></font>

In this exercise, we will use the adult_data_usa.csv data, that holds information about different people.
We will try to predict wheather a person earns more than 50K a year, or less than that.

We will use different feature selection techniques to select features for the models, and compare the recieved models.

# Questions

## load the dataset and explore it LIGHTLY

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

In [3]:
# your code here
df = pd.read_csv('adult_data_usa.csv')

In [4]:
df.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,income
5304,52,Private,218550,Some-college,10,Married-spouse-absent,Adm-clerical,Not-in-family,White,Female,14084,0,16,>50K
4022,46,Private,111979,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,47,<=50K
21784,39,?,361838,Bachelors,13,Married-civ-spouse,?,Wife,White,Female,0,0,6,>50K
15288,37,Private,223433,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,>50K
17195,28,Private,185127,Some-college,10,Never-married,Tech-support,Not-in-family,White,Male,0,0,54,<=50K


In [5]:
df.shape

(29170, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29170 entries, 0 to 29169
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             29170 non-null  int64 
 1   workclass       29170 non-null  object
 2   fnlwgt          29170 non-null  int64 
 3   education       29170 non-null  object
 4   education-num   29170 non-null  int64 
 5   marital-status  29170 non-null  object
 6   occupation      29170 non-null  object
 7   relationship    29170 non-null  object
 8   race            29170 non-null  object
 9   sex             29170 non-null  object
 10  capital-gain    29170 non-null  int64 
 11  capital-loss    29170 non-null  int64 
 12  hours-per-week  29170 non-null  int64 
 13  income          29170 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.1+ MB


In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,29170.0,38.655674,13.722408,17.0,28.00,37.0,48.0,90.0
fnlwgt,29170.0,187069.021152,104877.981749,12285.0,115895.25,176730.0,234138.5,1484705.0
education-num,29170.0,10.169455,2.394966,1.0,9.00,10.0,12.0,16.0
capital-gain,29170.0,1089.229928,7381.898528,0.0,0.00,0.0,0.0,99999.0
capital-loss,29170.0,88.510593,405.681709,0.0,0.00,0.0,0.0,4356.0
hours-per-week,29170.0,40.447755,12.417203,1.0,40.00,40.0,45.0,99.0


In [8]:
df.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
income            0
dtype: int64

## Randomly split the data into train and test sets (no need for validation at this point)

In [9]:
# your code here
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns = ['income']), df.income, test_size = 0.2, random_state = 42)

In [10]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((23336, 13), (23336,), (5834, 13), (5834,))

## Encode all categorical features, and turn the dependent variable into a numeric feature as well

In [11]:
ohe = OneHotEncoder()
ohe.fit(X_train[['workclass','marital-status','occupation','relationship','race','sex']])
X_train_temp = ohe.transform(X_train[['workclass','marital-status','occupation','relationship','race','sex']])
X_train_transformed = pd.DataFrame(X_train_temp.toarray(), columns = ohe.get_feature_names_out(), index = X_train.index)
X_test_temp = ohe.transform(X_test[['workclass','marital-status','occupation','relationship','race','sex']])
X_test_transformed = pd.DataFrame(X_test_temp.toarray(), columns = ohe.get_feature_names_out(), index = X_test.index)

In [12]:
X_train = pd.concat([X_train,X_train_transformed], axis = 1).drop(columns = ['workclass','marital-status','occupation','relationship','race','sex'])
X_test = pd.concat([X_test,X_test_transformed], axis = 1).drop(columns = ['workclass','marital-status','occupation','relationship','race','sex'])

In [13]:
X_train.shape,X_test.shape

((23336, 51), (5834, 51))

In [14]:
X_train.sample(5)

,age,fnlwgt,education,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male
10342,63,50349,HS-grad,9,0,0,34,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3466,46,142828,HS-grad,9,0,0,40,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
13020,39,101073,Assoc-acdm,12,0,0,24,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
21479,29,210959,Assoc-acdm,12,0,0,40,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
16397,26,269060,Bachelors,13,0,0,40,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [15]:
oe = OrdinalEncoder(categories = [[' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',
                                   ' 12th',' HS-grad',' Some-college',' Assoc-voc',' Assoc-acdm',
                                   ' Bachelors',' Masters',' Prof-school',' Doctorate']])
oe.fit(X_train[['education']])
X_train_education_temp = oe.transform(X_train[['education']])
X_train_ed_trans = pd.DataFrame(X_train_education_temp, columns = ['education_sklearn'], index = X_train.index)



In [16]:
X_train = pd.concat([X_train,X_train_ed_trans], axis = 1).drop(columns = 'education')

In [17]:
X_test_education_temp = oe.transform(X_test[['education']])
X_test_ed_trans = pd.DataFrame(X_test_education_temp, columns = ['education_sklearn'], index = X_test.index)


In [18]:
X_test = pd.concat([X_test,X_test_ed_trans], axis = 1).drop(columns = 'education')

In [19]:
X_train.sample()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,education_sklearn
26633,50,124793,10,0,0,55,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,9.0


In [20]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

## Train a model of your choice, and evaluate it. What are your recall and precision?

In [21]:
# your code here
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)


In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4403
           1       0.73      0.61      0.67      1431

    accuracy                           0.85      5834
   macro avg       0.81      0.77      0.78      5834
weighted avg       0.84      0.85      0.84      5834



For class 0, we get a precision of 0.88 and a recall of 0.93

For class 1, we get a precision of 0.73 and a recall of 0.62

## Try to select features with the Chi-square method, and one other filter method. Indicate which features were eliminated. Train the same model with the new subset of features and report your precision and recall

#### Chi-square method

In [23]:
# your code here
X_train_cat = X_train.drop(columns = ['age','fnlwgt','hours-per-week'])
kbest = SelectKBest(score_func = chi2, k = 35)
kbest.fit(X_train_cat,y_train)
X_train_chi2 = pd.DataFrame(kbest.transform(X_train_cat), columns = kbest.get_feature_names_out(), index = X_train.index)



In [24]:
X_test_cat = X_test.drop(columns = ['age','fnlwgt','hours-per-week'])
X_test_chi2 = pd.DataFrame(kbest.transform(X_test_cat), columns = kbest.get_feature_names_out(), index = X_test.index)

In [25]:
assert (X_test_chi2.columns.all() == X_train_chi2.columns.all())

In [26]:
X_train_chi2.sample()

,education-num,capital-gain,capital-loss,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,marital-status_ Divorced,marital-status_ Married-civ-spouse,...,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Black,race_ White,sex_ Female,sex_ Male,education_sklearn
3749,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,12.0


In [27]:
set(X_train_cat.columns) - set(X_train_chi2.columns)

{'marital-status_ Married-AF-spouse',
 'occupation_ Armed-Forces',
 'occupation_ Craft-repair',
 'occupation_ Priv-house-serv',
 'occupation_ Sales',
 'occupation_ Tech-support',
 'occupation_ Transport-moving',
 'race_ Asian-Pac-Islander',
 'race_ Other',
 'workclass_ Never-worked',
 'workclass_ Self-emp-not-inc',
 'workclass_ State-gov',
 'workclass_ Without-pay'}

All of the above features have been dropped.

#### f-regression method

In [28]:
X_train_num = X_train[['age','fnlwgt','education-num','hours-per-week']]
k_best_fs = SelectKBest(score_func = f_regression,k = 2)
k_best_fs.fit(X_train_num,y_train)
X_train_fs = pd.DataFrame(k_best_fs.transform(X_train_num), columns = k_best_fs.get_feature_names_out(), index = X_train.index)

In [29]:
X_test_num = X_test[['age','fnlwgt','education-num','hours-per-week']]
X_test_fs = pd.DataFrame(k_best_fs.transform(X_test_num), columns = k_best_fs.get_feature_names_out(), index = X_test.index)

In [30]:
assert( X_train_fs.columns.all() == X_test_fs.columns.all())

In [31]:
set(X_train_num.columns) - set(X_train_fs.columns)

{'fnlwgt', 'hours-per-week'}

'fnlwgt' and 'hours-per-week' have been dropped

In [32]:
X_train_dep = pd.concat([X_train_fs,X_train_chi2], axis = 1)
X_test_dep = pd.concat([X_test_fs,X_test_chi2], axis = 1)

In [33]:
rfc2 = RandomForestClassifier()
rfc2.fit(X_train_dep,y_train)
y_pred2 = rfc2.predict(X_test_dep)

print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      4403
           1       0.71      0.62      0.66      1431

    accuracy                           0.84      5834
   macro avg       0.79      0.77      0.78      5834
weighted avg       0.84      0.84      0.84      5834



We get slightly lower result than before.

We get for class 0 a precision of 0.88 and a recall of 0.91

We get for class 1 a precision of 0.70 and a recall of 0.63

## Try to select features with ONE wrapper method. Indicate which features were eliminated. Train the same model with the new subset of features and report your precision and recall

**Indicate how many features you kept, and PROVIDE CONCLUSIONS**

In [34]:
# your code here
rfc3 = RandomForestClassifier(n_jobs = -1)
rfc3.fit(X_train,y_train)


RandomForestClassifier(n_jobs=-1)

In [35]:
sfs = SequentialFeatureSelector(rfc3,n_features_to_select = 0.8, direction = 'backward', n_jobs = -1, cv = 3)
sfs.fit(X_train,y_train)


SequentialFeatureSelector(cv=3, direction='backward',
                          estimator=RandomForestClassifier(n_jobs=-1),
                          n_features_to_select=0.8, n_jobs=-1)

In [36]:
columns_left = list(X_train.columns[sfs.get_support()])
columns_removed = list(X_train.columns[~sfs.get_support()])

In [37]:
columns_removed

['education-num',
 'workclass_ ?',
 'workclass_ Federal-gov',
 'marital-status_ Married-AF-spouse',
 'marital-status_ Married-spouse-absent',
 'occupation_ ?',
 'occupation_ Priv-house-serv',
 'occupation_ Prof-specialty',
 'occupation_ Protective-serv',
 'relationship_ Wife',
 'race_ Amer-Indian-Eskimo']

Features above were removed


In [38]:
X_train = X_train[columns_left]
X_test = X_test[columns_left]

In [39]:
rfc4 = RandomForestClassifier(n_jobs = -1)
rfc4.fit(X_train,y_train)
y_pred_sfs = rfc4.predict(X_test)

print(classification_report(y_test,y_pred_sfs))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      4403
           1       0.73      0.62      0.67      1431

    accuracy                           0.85      5834
   macro avg       0.81      0.77      0.79      5834
weighted avg       0.85      0.85      0.85      5834



Class 0 : we get a precision of 0.88 and a recall of 0.93

Class 1 : we get a precision of 0.73 and a recall of 0.62

## Please sumarize your results. Did things happened as you expected? Please elaborate.

- By keeping all features :

For class 0, we get a precision of 0.88 and a recall of 0.93

For class 1, we get a precision of 0.73 and a recall of 0.62

- Then by using chi-square method and f-regression method, we got

class 0 : precision = 0.88 , recall = 0.91

class 1 : precision = 0.70 , recall = 0.63

- Then by using forward selection, we got

class 0 : precision = 0.88 , recall = 0.93

class 1 : precision = 0.73 , recall = 0.62

It looks like we get the same results betwene keeping all the features and keeping only the 80% most relevant ones.
By using filtering methods, we get slightly lower result.

Results looks a bit surprising as we would have expect to get better result when we keep only relevant features. It shows that we need to take care when we use those kind of technics.

